# Dados de Entrada
* Selecione "Adicionar ao Drive"
  *   https://tinyurl.com/bigdata-b3
  *   [Descrição dos dados](https://drive.google.com/file/d/1wu1yfR1A0A0Tl8Jl1ZNNuowr88YEFtXS/view?usp=sharing)






## Acesso ao Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Setup

## Instalação de pacotes

In [ ]:
!apt-get update  > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install findspark pyspark

## Preparação do ambiente

In [ ]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.5.1-bin-hadoop3

In [ ]:
import findspark
findspark.init("/content/spark-3.5.1-bin-hadoop3")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

from datetime import datetime

appName = 'Big Data SQL'
master = 'local[*]' # O * quer dizer tantas threads quanto tiver disponível,
# é possível colocando um inteiro entre colchetes definir quantas threads usar

spark = SparkSession.builder     \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [ ]:
input_data = spark.sparkContext.textFile('/content/drive/My Drive/b3/COTAHIST*1995*')


In [ ]:
input_data.take(10)

# Dataframes Estruturados

## Criação de Dataframes

In [ ]:
def read_line(line) :
  tipo = line[0:2]
  if tipo == '01' :
    data=datetime.strptime(line[2:10], '%Y%m%d')
    codbdi=int(line[10:12])
    codneg=line[12:24].strip()
    tpmerc=int(line[24:27])
    especi=line[39:49].replace('*', '').replace(' ','')
    preabe=float(line[56:69])/100
    premax=float(line[69:82])/100
    premin=float(line[82:95])/100
    preult=float(line[108:121])/100
    return Row(data=data,codbdi=codbdi,codneg=codneg,
               tpmerc=tpmerc,especi=especi,preabe=preabe,
               premax=premax,premin=premin,preult=preult)

  else :
    return Row(data=None,codbdi=None,codneg=None,
               tpmerc=None,especi=None,preabe=None,
               premax=None,premin=None,preult=None)


In [ ]:
cotacoes = input_data.map(read_line).toDF()
cotacoes.printSchema()


In [ ]:
cotacoes.count()

In [ ]:
cotacoes.show(20)

In [ ]:
cotacoes.describe().show()

## Descarte de linhas com valores nulos

In [ ]:
cotacoes = cotacoes.na.drop()
cotacoes.show(100)

## Arquivos Parquet

In [ ]:
cotacoes.show()

In [ ]:
cotacoes.write.parquet("b3-95.parquet")

In [ ]:
#input_data = spark.sparkContext.textFile('/content/drive/My Drive/b3/*TXT') # -- essa carga é mais lenta do que ler o parquet coalescido
#cotacoes = input_data.map(read_line).toDF()
#cotacoes.count()
#cotacoes = cotacoes.na.drop()
cotacoes.coalesce(1).write.parquet("b3-95-unico.parquet") #ao coalescer peder-se a vantagem de paralelização
#geralmente se faz a coalescencia das partições para escrever o arquivo

In [ ]:
cotacoes = spark.read.parquet('/content/drive/My Drive/b3/cotahist.parquet')

In [ ]:
cotacoes.show(10)

In [ ]:
cotacoes.count()

## Registro para consultas SQL

In [ ]:
cotacoes.createOrReplaceTempView("cotacoes")

In [ ]:
res = spark.sql("SELECT * FROM cotacoes")


In [ ]:
res.show(100)

In [ ]:
cotacoes.count()

## Filtros

In [ ]:
# Filtro para selecionar somente mercado a vista (tpmerc==10)

cotacoes_av = cotacoes.filter(cotacoes.tpmerc == 10)
cotacoes_av.show(20)


In [ ]:
cotacoes_av.count()

In [ ]:
selecionado = cotacoes.filter(cotacoes.codneg == 'PETR3')
selecionado.show()

In [ ]:
# Filtro para:
# acoes ordinarias (ON), novo mercado (ONNM), ou preferenciais (PN)
# lote padrão (codbdi==2)

cotacoes_sel = cotacoes_av.filter((cotacoes.codbdi == 2) &
                                 ((cotacoes.especi == 'ON') |
                                  (cotacoes.especi == 'ONNM') |
                                  (cotacoes.especi == 'PN')))

cotacoes_sel.show(10)


In [ ]:
cotacoes_sel.count()

In [ ]:
data_pro_filtro = datetime.strptime('20200403', '%Y%m%d')

In [ ]:
data_pro_filtro

In [ ]:
cotacoes_av.filter(((cotacoes_av.data > data_pro_filtro) & (cotacoes_av.codneg == 'PETR3'))).show(10)

### Filtro com sintaxe SQL

In [ ]:
res = spark.sql("SELECT * \
                 FROM cotacoes \
                 WHERE tpmerc = 10 \
                 AND codbdi = 2  \
                 AND especi IN ('ON', 'ONNM', 'PN')")


In [ ]:
res.count()

In [ ]:
res.show()

## Funções de Agrupamento

In [ ]:
cotacoes_sel.printSchema()

In [ ]:
# maior valor de fechamento para cada papel
preult_max = cotacoes_sel.groupBy('codneg').max('preult')
#preult_max.collect()
preult_max.show(20)

In [ ]:
preult_max.filter(preult_max.codneg == 'PETR4').show()

In [ ]:
preult_max.printSchema()

In [ ]:
# ordenar do maior para menor valor

preult_max.sort(desc('max(preult)')).show(10)


In [ ]:
preult_max.sort('max(preult)').show(10)


In [ ]:
# preco medio de fechamento, ordenado por valor

preult_avg = cotacoes_sel.groupBy('codneg').avg('preabe')


In [ ]:
preult_avg.printSchema()

In [ ]:
preult_avg.sort(desc('avg(preabe)')).show(50)

In [ ]:
# Desvio padrão do preço de fechamento
std = cotacoes_sel.groupBy('codneg').agg(stddev('preult'))
std.printSchema()
std = std.sort(desc('stddev(preult)'))
std = std.na.drop()
std.show(50)


### Agrupamento com sintaxe SQL

In [ ]:
cotacoes_sel.createOrReplaceTempView("cotacoes_sel")


In [ ]:
res = spark.sql("SELECT codneg, MAX(preult) \
                 FROM cotacoes_sel \
                 GROUP BY codneg \
                 SORT BY MAX(preult) DESC")


In [ ]:
res.show()

## Operações entre colunas

In [ ]:
# Coluna nova: diferença entre valor de fechamento e abertura

cotacoes_var = cotacoes_sel.withColumn('difpre', col('preult') - col('preabe'))
cotacoes_var.printSchema()
cotacoes_var.show(10)


In [ ]:
cotacoes_var.sort('difpre').show(10)

In [ ]:
cotacoes_var_rel = cotacoes_sel.withColumn('difpre_rel', round(100*((col('preult') - col('preabe'))/col('preabe')),2))
cotacoes_var_rel.printSchema()
cotacoes_var_rel.show(10)

In [ ]:
cotacoes_var_rel.sort(desc('difpre_rel')).show(10)

### Operações entre colunas com sintaxe SQL

In [ ]:
res = spark.sql("SELECT *, (preult-preabe) AS difpre\
                 FROM cotacoes_sel")


In [ ]:
res.show()

## Conversão de Dataframe para Lista

In [ ]:
# diferença máxima para cada papel, ordenado, top 10

max_var = cotacoes_var.groupBy('codneg') \
          .max('difpre') \
          .sort(desc('max(difpre)'))

max_var.printSchema()


In [ ]:
l = max_var.take(10)
print(l)
for item in l :
    print(str(item[0])+", "+str(item[1]))

## Join

In [ ]:
preult_max.show(5)

In [ ]:
# fechamento máximo para cada papel, ordenado, top 10

preult_max = cotacoes_var.groupBy('codneg').max('preult')
preult_max = preult_max.sort(desc('max(preult)'))
preult_max.printSchema()
preult_max.show(10)



In [ ]:
cotacoes_sel.printSchema()


In [ ]:
cotacoes_sel.count()

In [ ]:
preult_max.printSchema()


In [ ]:
preult_max.count()

In [ ]:
# Renomeando colona para desambiguação no join
preult_max = preult_max.withColumnRenamed('codneg', 'codneg_max')
preult_max.printSchema()

preult_max.show(5)


In [ ]:
cotacoes_sel.printSchema()
cotacoes_sel.show(10)

In [ ]:
# join para encontrar linhas dos precos maximos

datas_premax = cotacoes_sel.join(preult_max,
    [  preult_max['codneg_max'] == cotacoes_sel['codneg']  ,
       preult_max['max(preult)'] == cotacoes_sel['preult']   ]) #múltimplas condições de join são passadas em lista

datas_premax.printSchema()
datas_premax.show(20)



In [ ]:
datas_premax.count()

### Join com sintaxe SQL

In [ ]:
res = spark.sql("WITH premaxtmp AS \
                      (\
                        SELECT codneg, MAX(preult) as premax \
                        FROM cotacoes_sel \
                        GROUP BY codneg \
                      )\
                SELECT c.codneg, c.data, p.premax\
                FROM cotacoes_sel c, premaxtmp p\
                WHERE c.codneg = p.codneg AND c.preult = p.premax" )


In [ ]:
res.show()

In [ ]:
res.count()

## Seleção de Colunas

In [ ]:
# select ordenado por preult
datas_premax.select('codneg','data','preult') \
    .sort(desc('preult')) \
    .show(20)


## Seleção com join



In [ ]:
cotacoes_var.printSchema()

In [ ]:
# Encontrar códigos dos papeis que tiveram variação grande (>25 ou <-25)

var_grandes = cotacoes_var.filter((cotacoes_var.difpre > 25) | (cotacoes_var.difpre < -25))
var_grandes.show()

In [ ]:
r = var_grandes.select('codneg')
r.show(10)

In [ ]:
r.count()

In [ ]:
papeis_var_grande = var_grandes.select('codneg').distinct()

papeis_var_grande.show()

In [ ]:
papeis_var_grande.count()

In [ ]:
# Filtrar o dataset para incluir apenas papeis que tiveram variações grandes (left semi join)

reg = cotacoes_var.join(
     papeis_var_grande,cotacoes_var.codneg ==  papeis_var_grande.codneg,"leftsemi")
reg.show()

In [ ]:
reg.sort('data').show(10)

In [ ]:
# Filtrar o dataset para excluir papeis que tiveram variações grandes (left anti join)

reg = cotacoes_var.join(
     papeis_var_grande,cotacoes_var.codneg ==  papeis_var_grande.codneg,"leftanti")
reg.show(truncate=False)

### Join com sintaxe SQL

In [ ]:
cotacoes_var.createOrReplaceTempView('cotacoes_var')

In [ ]:
res = spark.sql("WITH largediff AS \
                  ( \
                    SELECT DISTINCT codneg \
                    FROM cotacoes_var\
                    WHERE ((difpre > 25) OR (difpre < -25)) \
                  ) \
                 SELECT * \
                 FROM cotacoes_var c LEFT SEMI JOIN largediff d  \
                 ON d.codneg = c.codneg")

In [ ]:
res.show()

In [ ]:
res = spark.sql("WITH largediff AS \
                  ( \
                    SELECT DISTINCT codneg \
                    FROM cotacoes_var\
                    WHERE ((difpre > 25) OR (difpre < -25)) \
                  ) \
                 SELECT * \
                 FROM cotacoes_var c LEFT ANTI JOIN largediff d  \
                 ON d.codneg = c.codneg")

In [ ]:
res.show()

## Janelas e operações entre linhas

In [ ]:
from pyspark.sql import Window

w = Window.partitionBy('codneg').orderBy('data')

cotacoes_com_anterior = cotacoes_sel.withColumn('preult_ant', lag('preult', 1).over(w))

cotacoes_com_anterior.printSchema()
cotacoes_com_anterior.show(10)



In [ ]:
cotacoes_com_anterior.filter(cotacoes_com_anterior.codneg == 'PETR3').show()

In [ ]:
cotacoes_com_anterior.withColumn('diff_preult', col('preult') - col('preult_ant')).sort(desc('diff_preult')).show()

### Operações entre linhas com SQL

In [ ]:
res = spark.sql("SELECT *, \
                    LAG(preult, 1) \
                      OVER \
                      ( \
                        PARTITION BY codneg \
                        ORDER BY data \
                      ) \
                    AS preult_ant\
                 FROM cotacoes_sel")

In [ ]:
res.show()

In [ ]:
cotacoes_var = cotacoes_com_anterior.withColumn('preult_diff', col('preult') - col('preult_ant'))
cotacoes_var.show()

## Criação de colunas com user-defined functions



In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

@udf(returnType=FloatType())
def percent_diff(curr_price, prev_price):
  if curr_price and prev_price :
    return (curr_price-prev_price)/prev_price
  else :
    return 0

In [ ]:
res = cotacoes_var.withColumn('diffp', percent_diff("preult", "preult_ant"))


In [ ]:
res.show()